----------------------------------------------------
Modelagem de tópicos de documentos provenientes da base de dados do CPDOC.
----------------------------------------------------
Base de dados: Antonio Azeredo da Silveira.
-----------------------------
Número de documentos: 10.428
-----------------------------
Número de imagens: 66.938
-----------------------------

### Topic: Clustering documents

## Contents

<ul>
<li>[Stopwords, stemming, and tokenization](#Stopwords,-stemming,-and-tokenizing)
<li>[Tf-idf and document similarity](#Tf-idf-and-document-similarity)
<li>[K-means clustering](#K-means-clustering)
<li>[Multidimensional scaling](#Multidimensional-scaling)
<li>[Visualizing document clusters](#Visualizing-document-clusters)
<li>[Hierarchical document clustering](#Hierarchical-document-clustering)
<li>[Latent Dirichlet Allocation (LDA)](#Latent-Dirichlet-Allocation)
</ul>

But first, I import everything I am going to need up front

In [2]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import feature_extraction
import mpld3
import pyLDAvis
import pyLDAvis.gensim

%matplotlib inline

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead

Specifying the path to the files  

In [3]:
#datapath = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected' # path marcelo
datapath = "../textfiles-corrected"
outputs = "../outputs/"
import os

onlyfiles = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath,f))]

In [4]:
onlyfiles[0:4]

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


['txt_0007168_484.txt',
 'txt_0062976_89.txt',
 'txt_0023552_242.txt',
 'txt_0002560_270.txt']

In [5]:
symbols = [x for x in string.punctuation]

#### make list of txt contents of files

In [6]:
count = 0
percentil = int(len(onlyfiles)/100)
textos = []

for txt in onlyfiles[0:1000]:
    with open(os.path.join(datapath,txt), 'r', encoding='utf-8') as f:
        count += 1
        if count % percentil == 0: print(int(count/percentil),'% done')
        texto = f.read()
        texto = texto.split()
        #print(len(texto))
        texto = [p for p in texto if p not in symbols]
        #print(len(texto))
        texto = [p.strip(string.punctuation) for p in texto]
        #print(len(texto))
        texto = [p for p in texto if not p.isdigit()]
        #print(len(texto))
        texto = [p for p in texto if len(p)>1]
        #print(len(texto))
        #print(texto)
        textos.append(texto)
#print(textos)

1 % done


### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the synopses. First, I load [NLTK's](http://www.nltk.org/) list of English stop words. [Stop words](http://en.wikipedia.org/wiki/Stop_words) are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [7]:
# load nltk's English stopwords as variable called 'stopwords'
additional_words = ['mr',]
stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words
stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [8]:
#Latent Dirichlet Allocation implementation with Gensim
from gensim import corpora, models, similarities 

%time texts = [[word for word in text if word not in stopwords] for text in textos]

/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/configparser.py:81: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  'PID': str(os.getpid()),
Using gpu device 0: Quadro K4200 (CNMeM is disabled, cuDNN 5103)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/rsouza/python/3/venv/local/lib/python3.5/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


CPU times: user 2.01 s, sys: 0 ns, total: 2.01 s
Wall time: 2.01 s


In [9]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

1008


In [10]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [12]:
len(corpus)

1000

In [19]:
%time lda = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=10)

CPU times: user 2min 9s, sys: 68 ms, total: 2min 9s
Wall time: 2min 9s


In [20]:
lda.print_topics(5, num_words=20)

[(5,
  '0.005*brasil + 0.003*ministro + 0.002*países + 0.002*governo + 0.002*club + 0.001*república + 0.001*us + 0.001*presidente + 0.001*telegrama + 0.001*embaixador + 0.001*ser + 0.001*apartheid + 0.001*geral + 0.001*eh + 0.001*bons + 0.001*sr + 0.001*exteriores + 0.001*relações + 0.001*política + 0.001*visita'),
 (7,
  '0.005*países + 0.004*brasil + 0.003*governo + 0.003*desenvolvimento + 0.002*cooperação + 0.002*senhora + 0.002*relações + 0.002*ser + 0.002*nações + 0.002*unidas + 0.002*presidente + 0.002*ministro + 0.001*parte + 0.001*brasileiro + 0.001*internacional + 0.001*acordo + 0.001*dois + 0.001*exteriores + 0.001*telegrama + 0.001*geral'),
 (3,
  '0.004*brasil + 0.004*senhor + 0.003*ministro + 0.003*presidente + 0.003*exteriores + 0.003*excelência + 0.002*república + 0.002*ser + 0.002*governo + 0.002*países + 0.002*relações + 0.002*telegrama + 0.002*vossa + 0.002*brasileiro + 0.002*rio + 0.002*acordo + 0.002*embaixador + 0.001*nacional + 0.001*visita + 0.001*itaipu'),
 (6,


In [21]:
lda.show_topics()

[(0,
  '0.006*brasil + 0.004*relações + 0.002*exteriores + 0.002*países + 0.002*united + 0.002*governo + 0.002*states + 0.002*presidente + 0.002*acordo + 0.002*commission'),
 (1,
  '0.007*vg + 0.004*countries + 0.004*brasil + 0.003*ser + 0.003*senhor + 0.002*países + 0.002*presidente + 0.002*pt + 0.002*ministro + 0.002*telegrama'),
 (2,
  '0.005*brasil + 0.004*presidente + 0.004*ministro + 0.003*senhor + 0.003*ser + 0.003*governo + 0.002*países + 0.002*república + 0.002*embaixador + 0.002*relações'),
 (3,
  '0.004*brasil + 0.004*senhor + 0.003*ministro + 0.003*presidente + 0.003*exteriores + 0.003*excelência + 0.002*república + 0.002*ser + 0.002*governo + 0.002*países'),
 (4,
  '0.006*países + 0.005*brasil + 0.004*relações + 0.004*ministro + 0.002*desenvolvimento + 0.002*exteriores + 0.002*ser + 0.002*senhor + 0.002*governo + 0.002*presidente'),
 (5,
  '0.005*brasil + 0.003*ministro + 0.002*países + 0.002*governo + 0.002*club + 0.001*república + 0.001*us + 0.001*presidente + 0.001*tele

In [22]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)

In [23]:
topics_matrix[3]

(3,
 [('brasil', 0.0044031461499027622),
  ('senhor', 0.0042193573667761381),
  ('ministro', 0.0030219345549810442),
  ('presidente', 0.0028680800867773395),
  ('exteriores', 0.0028027741257134415),
  ('excelência', 0.0026452128229373715),
  ('república', 0.0024025642675266153),
  ('ser', 0.0022953930556888235),
  ('governo', 0.0022582958609677134),
  ('países', 0.0022462449920188695),
  ('relações', 0.0021789213982034423),
  ('telegrama', 0.0019408140708849857),
  ('vossa', 0.0019283287797634975),
  ('brasileiro', 0.0018897977420929643),
  ('rio', 0.0017726205640245117),
  ('acordo', 0.0016786014490598152),
  ('embaixador', 0.0015369928386683221),
  ('nacional', 0.0013780804977919659),
  ('visita', 0.0013694671077892513),
  ('itaipu', 0.0013662136771165105)])

In [27]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [26]:
pyLDAvis.save_html()

TypeError: save_html() missing 2 required positional arguments: 'data' and 'fileobj'